In [2]:
# imports
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# sklearn imports
from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# # tree classifier
# from sklearn.model_selection import train_test_split
# from sklearn.tree import DecisionTreeClassifier, plot_tree

# # rainforest classifier
# from sklearn.ensemble import RandomForestClassifier

# # linear regession classifier
# from sklearn.linear_model import LogisticRegression

# # knn classifier
# from sklearn.neighbors import KNeighborsClassifier
import wrangle as w
import explore as e
import model as m

# Project Description
The healthcare breach dataset is utilized in this project to find drivers for breach type. 


# Project Goal

* Discover drivers of breach type in the healthcare breach dataset 
* Use drivers to develop a machine learning model to classify healthcare facilities with the probablity of a breach type.
* Breach types are security incidents in which unauthorized parties gain access to sensitive data or confidential information.
* This information could be used to further our understanding of which elements contribute to a breach types.

# Initial Thoughts

My initial hypothesis is that drivers of breaches will be elements such as state, entity_type, season, location, mulit-breach location

# Acquire
* Aquire dataset from [Data.World]( 
* It contained 7043 rows and 26 columns before cleaning
* Each row represents a customer at Telco
* Each column represents a feature of those customers